In [1]:
import pandas as pd
import timeit
import requests
from bs4 import BeautifulSoup, NavigableString, Tag
import yaml
import base64

In [2]:
recipe_lst = []

recipe_lst.append(list(range(1000,12849)))

for y in [str(year) for year in list(range(1013,1023+1))]:
    for n in [str(recipe_no).zfill(3) for recipe_no in list(range(1, 999))]:
        recipe_lst.append(y + n)

# newest recipe first
recipe_lst = list(reversed(recipe_lst))

In [3]:
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

In [ ]:
for chunk in list(chunker(recipe_no, 250)):
    recipes = []
    
    for r in chunk:
        # soup
        link = f'https://cooking.nytimes.com/recipes/{r}'
        reqs = requests.get(link)
        soup = BeautifulSoup(reqs.text, 'html.parser')

        values = {}
        description = []
        ingredients = []
        directions = []
        note = []

        try:
            # description
            description.append(soup.find('div', attrs = {'class': 'recipeintro_topnote-block__hFFPr'}).find('p').text.strip())
            description.append('Rating: ' + soup.find('dd', attrs = {'class' : 'pantry--ui stats_ratingInfo__PUgd2'}).text.strip())

            # ingredients
            i_lst = soup.findAll('li', attrs = {'class' : 'pantry--ui ingredient_ingredient__lq70t'})
            for i in i_lst:
                if len(i.findAll('span')) > 1:
                    ingredients.append(i.findAll('span')[0].text.strip() + ' ' + i.findAll('span')[1].text.strip())
                elif len(i.findAll('span')) == 1:
                    ingredients.append(i.findAll('span')[0].text.strip())

            # preparation        
            p_lst = soup.find('div', attrs = {'class' : 'recipebody_prep-block__cpC8w'}).findAll('p')
            for i in p_lst:
                directions.append(i.text.strip())

            # note
            try:
                t_lst = soup.find('div', attrs = {'class' : 'tips_tips__iZQZC'}).findAll('li')
                for i in t_lst:
                    if len(t_lst) == 0:
                        pass
                    else:
                        note.append(i.text.strip())
            except AttributeError:
                pass

            # update values
            values.update(
                {'name' : soup.find('h1').text.strip(),
                 'servings' : soup.find('span', attrs = {'class' : 'pantry--ui ingredients_fontOverride__WoKY5'}).text.strip(),
                 'source':soup.findAll('h2')[0].find('a').text.strip(),
                 'source_url' : link,
                 'cook_time': soup.find('div', attrs = {'class' : 'recipeintro_stats-block__YgbqJ'}).findAll('dd')[0].text.strip(),
                 'nutritional_info' : soup.find('h5').next_sibling.text.strip(),
                 'description' : '\n\n'.join(description),
                 'ingredients' : '\n'.join(ingredients),
                 'directions' : '\n\n'.join(directions),
                 'notes' : '\n\n'.join(note)
                })

            # update image if available
            if soup.find('div', attrs = {'class' : 'recipeheaderimage_image___CZR1'}):
                values.update(
                    {'photo' : base64.b64encode(requests.get(soup.find('div', attrs = {'class' : 'recipeheaderimage_image___CZR1'}).find('img')['src'].split('?')[0]).content).decode('ascii')
                    })

            recipes.append(values)

        except AttributeError:
            pass

    recipe_no_range = str(chunk[0]) + '-' + str(chunk[-1])

    # export it to YAML file
    with open(f"../data/recipes_{recipe_no_range}.yml", "w", encoding = 'utf-8') as file:
        yaml.dump(recipes, file, width=10000)
        
    stop = timeit.default_timer()

    print(recipe_range + '_Finished ' + timeit.default_timer())